# Data cleanup
This will outline the process that I am choosing along with the reasons why
I have chosen to further remove variables from my dataset. Along with the 
interpolation methods that I have decided to use to deal with data that is missing
in the Training and Validation datasets.

Based on year 2007 minute data I have to drop the following variables, reason included.
visible_cloud_optical_depth: 432899 missing values out of 525567, 82.4% missing data
cloud_radiating_temperature: 174490 missing values out of 525567, 33.2% missing data


In [ ]:
import pandas as pd
import numpy as np
# using 2007 as first example. Will check all years
df = pd.read_csv("/home/nelson/PycharmProjects/Solar Forecasting Thesis Project/"
                 "Data/train/2007/minute_data_total_year.csv", index_col="time")
df.isnull().sum()  # sums up the missing data 

count = 0
index_holder = 0
i = 0
while i < len(df.index) :
        if np.isnan(df["downwelling_shortwave"].iloc[i]):
            for j in range(i, len(df.index)):
                if count == 0:
                    index_holder = i
                if not np.isnan(df["downwelling_shortwave"].iloc[j]):
                    count = 0
                    index_holder = 0
                    break
                count += 1
                if count == 10:
                    print("Starts at index " + df.index[index_holder])
                    print("10 missing at " + str(df.index[j]))
                i += 1
            count = 0
            index_holder = 0
        i += 1
            